In [1]:
import pandas as pd
import pymongo
import numpy as np
import os

In [2]:
client = pymongo.MongoClient()
db = client.swatford

In [99]:
def str_split(x,y):
    if x is not None:
        return x.split(y)
    else:
        return []

In [131]:
data = {"gene_id":[],"pmid":[],"human":[]}

In [132]:
for fn in os.listdir("../data/RGD/"):
    h = "N"
    if "HUMAN" in fn:
        h="Y"
    x = pd.read_table(f"../data/RGD/{fn}",
                  comment="#",
                  low_memory=False,
                  converters={"CURATED_REF_PUBMED_ID":str,
                         "UNCURATED_PUBMED_ID":str})[["NCBI_GENE_ID",
                                                      "CURATED_REF_PUBMED_ID",
                                                      "UNCURATED_PUBMED_ID"]].copy().replace({"":None})
    for tup in x[x.NCBI_GENE_ID.isnull()==False].itertuples():
        pmids = [int(y) for y in str_split(tup.CURATED_REF_PUBMED_ID,";") + str_split(tup.UNCURATED_PUBMED_ID,";")]
        data["gene_id"] += [int(tup.NCBI_GENE_ID)]*len(pmids)
        data["pmid"] += pmids
        data["human"] += [h]*len(pmids)
    print(fn)

GENES_RAT.txt
GENES_HUMAN.txt
GENES_MOUSE.txt
GENES_BONOBO.txt
GENES_SQUIRREL.txt
GENES_CHINCHILLA.txt
GENES_DOG.txt


In [133]:
to_insert = []
for i,row in pd.DataFrame(data).iterrows():
    to_insert.append(dict(row))
    if i%25000==0:
        db.rgd.insert_many(to_insert)
        to_insert=[]
db.rgd.insert_many(to_insert)